<h1 align="center" style="font-family:'Georgia'; font-weight:bold;padding:2%; font-size:50px">Bees, Wasps or Other Insects</h1>
<hr>
<h1 style="font-family:'Georgia'; font-weight:bold;padding:1%">Imports</h1>

In [97]:
import os
import PIL
import cv2
import math
import librosa
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, \
recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, \
classification_report, log_loss ,precision_recall_curve, balanced_accuracy_score

<h1 style="font-family:'Georgia'; font-weight:bold;padding:1%">Constants and Global Variables</h1>

In [98]:
DATASET_PATH = '../Datasets/On Door/'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
SAMPLE_RATE = 22050

In [99]:
models = {
    'buzzingDetection':layers.SimpleRNN, 
    'insectDetection': None, 
    'beeOrNot': None, 
    'waspOrNot': None, 
    'decisionMaking': None
    }
datasets = ['BuzzOrNot', 'BeeOrNot', 'WaspOrNot']

<h1 style="font-family:'Georgia'; font-weight:bold;padding:1%">Evaluation</h1>

In [100]:
def evaluateModel(yTrue, yPred, yProb=None):
    accuracy = accuracy_score(yTrue, yPred)
    print(f"Accuracy: {accuracy:.4f}")
    
    balancedAcc = balanced_accuracy_score(yTrue, yPred)
    print(f"Balanced Accuracy: {balancedAcc:.4f}")
    
    precision = precision_score(yTrue, yPred)
    print(f"Precision: {precision:.4f}")
    
    recall = recall_score(yTrue, yPred)
    print(f"Recall: {recall:.4f}")
    
    f1 = f1_score(yTrue, yPred)
    print(f"F1 Score: {f1:.4f}")
    
    cm = confusion_matrix(yTrue, yPred)
    print(f"Confusion Matrix:\n{cm}")
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    if yProb is not None:
        rocAuc = roc_auc_score(yTrue, yProb)
        print(f"ROC AUC Score: {rocAuc:.4f}")
        
        fpr, tpr, _ = roc_curve(yTrue, yProb)
        plt.figure(figsize=(6, 4))
        plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {rocAuc:.4f})")
        plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate (Recall)")
        plt.title("ROC Curve")
        plt.legend(loc="lower right")
        plt.show()
        
        precisionCurve, recallCurve, _ = precision_recall_curve(yTrue, yProb)
        plt.figure(figsize=(6, 4))
        plt.plot(recallCurve, precisionCurve, label="Precision-Recall Curve")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title("Precision-Recall Curve")
        plt.legend(loc="lower left")
        plt.show()

    if yProb is not None:
        logloss = log_loss(yTrue, yProb)
        print(f"Log Loss: {logloss:.4f}")

<h1 align="center" style="font-family:'Georgia'; font-weight:bold;padding:1%">Buzz Detection (Sound Classification)</h1>

---

<h2 style="font-family:'Georgia'; font-weight:bold;padding:1%">Audio Segmentation</h2>

In [101]:
def loadAudio(sub='/train'):
    print(f"\nLoading audio data from {DATASET_PATH + datasets[0] + sub}...")
    PATH = DATASET_PATH + datasets[0] + sub
    segments = []
    targets = []
    
    for filename in os.listdir(PATH):
        if filename.endswith('.wav'):
            audio = os.path.join(PATH, filename)
            label = filename.split('.')[0].split('_')[0]
            try:
                audio, sampleRate = librosa.load(audio)
                segments.append(audio)
                targets.append('Buzzing' if filename.startswith( 'Bee') else 'No Buzzing')
            except:
                os.remove(audio)
  
    return segments, np.array(targets)       

<h2 style="font-family:'Georgia'; font-weight:bold;padding:1%">Audio Pre-Proccessing</h2>

In [102]:
def extractMFCC(segments, n_mfcc=12):
    print(f"\nPadded MFCC extraction for {len(segments)} segments...")
    mfccs = []
    mfccs = [librosa.feature.mfcc(y=segment, sr=SAMPLE_RATE, n_mfcc=n_mfcc) for segment in segments]
    return mfccs, max([mfcc.shape[1] for mfcc in mfccs])


In [103]:
def pad(mfccs, maxLength):
    features = [
        np.pad(mfcc, ((0, 0), (0, maxLength - mfcc.shape[1])), mode='constant') 
        for mfcc in mfccs
    ]
    return np.array(features)

<h1 align="center" style="font-family:'Georgia'; font-weight:bold;padding:1%">Bee Detection (Image Classification)</h1>
<hr>
<h2 style="font-family:'Georgia'; font-weight:bold;padding:1%">Image Pre-Proccessing</h2>

In [105]:
def loadImages(index, sub='/train'):
    print(f"\nLoading image data from {DATASET_PATH + datasets[index] + sub}...")
    PATH = DATASET_PATH + datasets[1] + sub
    subDirs = [d for d in os.listdir(PATH) if os.path.isdir(os.path.join(PATH, d))]
    targets = []
    images = []
    for target in subDirs:
        targetPath = os.path.join(PATH, target)
        
        for filename in os.listdir(targetPath):
            
            if filename.endswith(('.jpg', '.png')): 
                image = os.path.join(targetPath, filename)
                img = cv2.imread(image)
                img = cv2.resize(img, IMAGE_SIZE) 
                targets.append(target)
                images.append(img)
                  
    return images, np.array(targets)

In [106]:
def imagePipeline(index):
    xTrain, yTrain = loadImages(index, '/train/')
    xTest, yTest = loadImages(index, '/test/')
    xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size=0.2, random_state=42)
    return xTrain, yTrain, xVal, yVal, xTest, yTest

<h1 style="font-family:'Georgia'; font-weight:bold;padding:1%">Model Building</h1>

In [107]:
def buildModel(shape, modelType):  
        print(f"\nBuilding {modelType.__name__} with input shape {shape}...")
        model = Sequential()
        model.add(layers.Input(shape=shape))
        
        model.add(modelType(128, return_sequences=True))
        model.add(modelType(64))
        #model.add(layers.Dense(64, activation='tanh'))

        
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        return model

In [108]:
def train(model, xTrain, yTrain, xVal, yVal):
    print(f"\nTraining {model.layers[0].__class__.__name__} model...")
    model.fit(xTrain, yTrain, validation_data=(xVal, yVal), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    predictions = model.predict(xVal)
    predictions = np.array([1 if pred > 0.5 else 0 for pred in predictions])
    evaluateModel(yVal, predictions)
    return model

In [109]:
def test(model, xTest, yTest):
    predictions = model.predict(xTest)
    predictions = np.array([1 if pred > 0.5 else 0 for pred in predictions])
    evaluateModel(yTest, predictions)

In [110]:
def audioPipeline():
    xTrain, yTrain = loadAudio('/train/')
    xTest, yTest = loadAudio('/test/')
    xTrain, maxTrainLength = extractMFCC(xTrain)
    xTest, maxTestLength = extractMFCC(xTest)
    xTrain = pad(xTrain, max(maxTrainLength, maxTestLength))
    xTest = pad(xTest, max(maxTrainLength, maxTestLength))
    yTrain = np.array([1 if label == 'Buzzing' else 0 for label in yTrain])
    yTest = np.array([1 if label == 'Buzzing' else 0 for label in yTest])
    xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size=0.2, random_state=42)
    return xTrain, yTrain, xVal, yVal, xTest, yTest

<h1 align="center" style="font-family:'Georgia'; font-weight:bold;padding:2%">Main</h1>

In [111]:
def compareAudioModels(): 
    xTrain, yTrain, xVal, yVal, xTest, yTest = audioPipeline()
    models = [layers.LSTM, layers.GRU, layers.SimpleRNN]
    for model in models:
        model = buildModel(xTrain.shape[1:], model)
        model = train(model, xTrain, yTrain, xVal, yVal)
        test(model, xTest, yTest)
        
# compareAudioModels()

In [112]:
xTrain, yTrain, xVal, yVal, xTest, yTest = imagePipeline(1)


Loading image data from ../Datasets/On Door/BeeOrNot/train/...


KeyboardInterrupt: 